In [9]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

In [1]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

220922-13:31:25,256 nipype.utils WARNING:
	 A newer version (1.8.4) of nipy/nipype is available. You are using 1.7.0


### Copes at level 2:
##### Model 0:
1. con
2. simon
3. flanker
4. inc
5. simon-con
6. flanker-con
7. inc-con
8. simon-flanker
9. inc-simon
10. inc-flanker

##### Model 1:
1. response_index
2. response_middle
3. response_ring
4. response_index - response_middle
5. response_index - response_ring
6. response_middle - response_ring

##### Model 2:
1. Correct
2. Incorrect
3. Incorrect - Correct

In [2]:
# general set-up
project_folder = '/home/Public/trondheim'
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
# The directory where we can find the level 2 output
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model')


spaces = ['T1w']   # shouldn't touch this but just in case we _do_ want to go back to MNI....
ses = 'sstmsit'      # don't make this a list, that won't work
task = 'msit'     # don't make this a list, that won't work
model_ns = ['0']      # 
fwhms = ['1p5']#, '4p5']

template_brain = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
template_brain_mask = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii'

subject_ids = sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', 'sub-*', f'ses-{ses}', 'func','fwhm-1p5',f'model-{model_ns[0]}',f'*task-{task}_space-MNI*contrast-0_desc-zstat*')))
subject_ids = [x.split('/')[-6].split('-')[-1] for x in subject_ids]

if model_ns[0] == '0':
#     contrasts = ['0']
    contrasts = ['0','1','2','3','4','5','6','7','8','9']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2','3','4','5'] # motor from second level model
elif model_ns[0] == '2':
    contrasts = ['0','1','2']

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', f'ses-{ses}', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}

## if uneven number of template / designs / subs the code will crash
print(f"no.copes = {len(glob.glob(templates['level2_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")
print(f"no.varvopes = {len(glob.glob(templates['level2_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")
print(f"no.tdofs = {len(glob.glob(templates['level2_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}")



no.copes = 36
no.varvopes = 36
no.tdofs = 36


In [3]:
msitsubs = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', 'sub-*', 'ses-sstmsit', 'func','fwhm-1p5',f'model-{model_ns[0]}','*task-msit_space-MNI*contrast-0_desc-zstat*')))
print(f'Group level based on {len(msitsubs)} subjects')

msit = [x.split('/')[-6].split('-')[-1] for x in msitsubs]
allmsit = [x.split('/')[-2].split('-')[-1] for x in sorted(glob.glob(os.path.join('/home/Public/trondheim', 'sourcedata', 'zipdata', 'sub-*', 'ses-sstmsit')))]
print(f'subs {list(set(allmsit) - set(msit))} are missing')

print(len(subject_ids))  # should be 36 as sub 025 removed

Group level based on 36 subjects
subs ['025'] are missing
36


In [4]:
##
workflow = pe.Workflow(name='feat_level3_msit_hp')
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [5]:
def get_subject_ids(): # get ids of subjects that have subject_level_model data for this model
    import glob
    import os
    subject_ids = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', 'sub-*', 'ses-sstmsit', 'func','fwhm-1p5','model-0','*task-msit_space-MNI*contrast-0_desc-zstat*')))
    subject_ids = [x.split('/')[-6].split('-')[-1] for x in subject_ids]
    print(f'subs are: {subject_ids}')
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    print(f'Number of subjects included in design matrix: {len(subject_ids)}')
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')

In [6]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [7]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [8]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-msit",f"model-{model_ns[0]}")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [9]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", "ses-sstmsit","task-msit",f"model-{model_ns[0]}",f"model-{model_ns[0]}")

substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                        (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [ ]:
# uses 2 cores per process
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 20, 'memory_gb': 200})

220922-13:32:01,720 nipype.workflow INFO:
	 Workflow feat_level3_msit_hp settings: ['check', 'execution', 'logging', 'monitoring']
220922-13:32:02,39 nipype.workflow INFO:
	 Running in parallel.
220922-13:32:02,117 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 12 jobs ready. Free memory (GB): 200.00/200.00, Free processors: 20/20.
220922-13:32:02,151 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_msit_hp.subject_id_getter).
220922-13:32:02,152 nipype.workflow INFO:
	 [Job 1] Cached (feat_level3_msit_hp.get_volume).
220922-13:32:02,199 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_msit_hp.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/selector".
220922-13:32:02,201 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_msit_hp.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_1_fwhm_1p5_model_n_0/selector".
220922-13:32:02,201 nipyp

220922-13:32:06,164 nipype.workflow INFO:
	 [Job 35] Cached (feat_level3_msit_hp.varcopemerge).
220922-13:32:06,170 nipype.workflow INFO:
	 [Job 36] Cached (feat_level3_msit_hp.copemerge).
220922-13:32:06,172 nipype.workflow INFO:
	 [Job 37] Cached (feat_level3_msit_hp.varcopemerge).
220922-13:32:06,175 nipype.workflow INFO:
	 [Job 38] Cached (feat_level3_msit_hp.copemerge).
220922-13:32:06,177 nipype.workflow INFO:
	 [Job 39] Cached (feat_level3_msit_hp.varcopemerge).
220922-13:32:06,180 nipype.workflow INFO:
	 [Job 40] Cached (feat_level3_msit_hp.copemerge).
220922-13:32:06,182 nipype.workflow INFO:
	 [Job 41] Cached (feat_level3_msit_hp.varcopemerge).
220922-13:32:06,201 nipype.workflow INFO:
	 [Job 42] Cached (feat_level3_msit_hp.l3model).
220922-13:32:06,205 nipype.workflow INFO:
	 [Job 43] Cached (feat_level3_msit_hp.l3model).
220922-13:32:06,209 nipype.workflow INFO:
	 [Job 44] Cached (feat_level3_msit_hp.l3model).
220922-13:32:06,227 nipype.workflow INFO:
	 [Job 45] Cached (fea

220922-13:32:08,443 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_msit_hp.flameo" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_7_fwhm_1p5_model_n_0/_run_mode_flame12/flameo".
220922-13:32:08,443 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level3_msit_hp.flameo".
220922-13:32:08,453 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
220922-13:32:08,453 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
220922-13:32:08,454 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
220922-13:32:08,469 nipype.workflow INFO:
	 [Node] Outdated cache found for "feat_level3_msit_hp.flameo".
220922-13:32:08,471 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_msit_hp.flameo" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_8_fwhm_1p5_model_n_0/_run_mode_flame1/flameo".
22

220922-13:39:12,576 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate1" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_2_fwhm_1p5_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate1".
220922-13:39:12,624 nipype.workflow INFO:
	 [Node] Executing "_smoothestimate1" <nipype.interfaces.fsl.model.SmoothEstimate>
220922-13:39:13,331 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 0.705341s.
220922-13:39:14,322 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 426.017922s.
220922-13:39:14,525 nipype.workflow INFO:
	 [Job 54] Completed (feat_level3_msit_hp.flameo).
220922-13:39:14,527 nipype.workflow INFO:
	 [Job 173] Completed (_smoothestimate1).
220922-13:39:14,529 nipype.workflow INFO:
	 [MultiProc] Running 18 tasks, and 2 jobs ready. Free memory (GB): 196.40/200.00, Free processors: 2/20.
                     Currently running:
                       * feat_level3_msit_hp.flameo
   

220922-13:39:20,529 nipype.workflow INFO:
	 [Job 74] Completed (feat_level3_msit_hp.smoothestimate).
220922-13:39:20,531 nipype.workflow INFO:
	 [Job 176] Completed (_grf_cluster0).
220922-13:39:20,533 nipype.workflow INFO:
	 [MultiProc] Running 18 tasks, and 5 jobs ready. Free memory (GB): 196.40/200.00, Free processors: 2/20.
                     Currently running:
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
      

...

220922-13:40:02,684 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_7_fwhm_1p5_model_n_0/_run_mode_flame1/_threshold_3.1/_grf_cluster1/zstat2_threshold.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-7_grouplevelcontrast-2_flame-1_desc-zstat_voxelthreshold-3.1.nii.gz
220922-13:40:02,873 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/level3_zstats/_contrast_n_7_fwhm_1p5_model_n_0/_run_mode_flame1/zstat1.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-7_grouplevelcontrast-1_flame-1_desc-zstat.nii.gz
220922-13:40

220922-13:40:05,30 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/level3_zstats/_contrast_n_4_fwhm_1p5_model_n_0/_run_mode_flame1/zstat1.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-4_grouplevelcontrast-1_flame-1_desc-zstat.nii.gz
220922-13:40:05,31 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/level3_zstats/_contrast_n_4_fwhm_1p5_model_n_0/_run_mode_flame1/zstat2.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-4_grouplevelcontrast-2_flame-1_desc-zstat.nii.gz
220922-13:40:05,32 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/gl

220922-13:40:10,335 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 481.913642s.
220922-13:40:10,569 nipype.workflow INFO:
	 [Job 62] Completed (feat_level3_msit_hp.flameo).
220922-13:40:10,571 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 1 jobs ready. Free memory (GB): 198.00/200.00, Free processors: 10/20.
                     Currently running:
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
                       * feat_level3_msit_hp.flameo
220922-13:40:12,572 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 2 jobs ready. Free memor

220922-13:40:18,683 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
220922-13:40:18,683 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster0" <nipype.interfaces.fsl.model.Cluster>
220922-13:40:18,708 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
220922-13:40:18,708 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
220922-13:40:19,762 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 1.077426s.
220922-13:40:19,836 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.125929s.
220922-13:40:19,903 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster0", elapsed time 1.219203s.
220922-13:40:20,40 nipype.workflow INFO:
	 [Node] Finished "_grf_cluster1", elapsed time 1.3297919999999999s.
220922-13:40:20,578 nipype.workflow INFO:
	 [Job 228] Completed (_grf_cluster0).
220922-13:40:20,579 nipype.workflow INFO:
	 [J

220922-13:40:22,990 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/level3_zstats/_contrast_n_5_fwhm_1p5_model_n_0/_run_mode_flame1/zstat1.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-5_grouplevelcontrast-1_flame-1_desc-zstat.nii.gz
220922-13:40:22,991 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/third_level_model/level3_zstats/_contrast_n_5_fwhm_1p5_model_n_0/_run_mode_flame1/zstat2.nii.gz -> /home/Public/trondheim/derivatives/glm_feat_hp_sct/group_level_model/ses-sstmsit/task-msit/model-0/model-0/model-0/model-0_fwhm-1p5_subjectlevelcontrast-5_grouplevelcontrast-2_flame-1_desc-zstat.nii.gz
220922-13:40:22,992 nipype.interface INFO:
	 sub: /home/Public/trondheim/derivatives

In [15]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/_run_mode_flame1/flameo/_inputs.pklz')
res

{'cope_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/copemerge/sub-002_ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-0_contrast-0_desc-cope_merged.nii.gz',
 'var_cope_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/varcopemerge/sub-002_ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-0_contrast-0_desc-varcope_merged.nii.gz',
 'mask_file': '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii',
 'design_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/l3model/design.mat',
 't_con_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/l3model/design.con',
 'cov_split_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_

In [16]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/_run_mode_flame1/flameo/_inputs.pklz')
res

{'cope_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/copemerge/sub-002_ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-0_contrast-5_desc-cope_merged.nii.gz',
 'var_cope_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/varcopemerge/sub-002_ses-sstmsit_task-msit_space-MNI152NLin2009cAsym_model-0_contrast-5_desc-varcope_merged.nii.gz',
 'mask_file': '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii',
 'design_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/l3model/design.mat',
 't_con_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/l3model/design.con',
 'cov_split_file': '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_

In [26]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_5_fwhm_1p5_model_n_0/l3model/_inputs.pklz')
res

{'contrasts': [('Group mean', 'T', ['intercept'], [1.0]),
  ('-Group mean', 'T', ['intercept'], [-1.0])],
 'regressors': {'intercept': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0]}}

In [27]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_msit_hp/_contrast_n_0_fwhm_1p5_model_n_0/l3model/_inputs.pklz')
res

{'contrasts': [('Group mean', 'T', ['intercept'], [1.0]),
  ('-Group mean', 'T', ['intercept'], [-1.0])],
 'regressors': {'intercept': [1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0]}}

In [28]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_005/modelgen/_inputs.pklz')
res

{'args': ['/home/Public/trondheim/derivatives/confounds/sub-005/ses-sstmsit/func/sub-005_ses-sstmsit_task-msit_run-1_desc-confounds_timeseries.txt',
  '/home/Public/trondheim/derivatives/confounds/sub-005/ses-sstmsit/func/sub-005_ses-sstmsit_task-msit_run-2_desc-confounds_timeseries.txt'],
 'environ': {'FSLOUTPUTTYPE': 'NIFTI_GZ'},
 'ev_files': [['/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_005/level1design/ev_con_0_1.txt',
   '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_005/level1design/ev_simon_0_3.txt',
   '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_005/level1design/ev_flanker_0_5.txt',
   '/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_005/level1

In [29]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/nipype_workflow_folders/feat_level12_msit_hp/_model_n_0_smoothing_fwhm_1p5_space_T1w_subject_id_002/level1design/_inputs.pklz')
res

{'interscan_interval': 1.38,
 'session_info': [{'cond': [{'name': 'con',
     'onset': [11.394646017057793,
      25.198881346210808,
      46.9056121231429,
      52.807448627138484,
      80.41586769599465,
      107.5242528670374,
      114.92655728099864,
      122.32880041509858,
      143.03511309315454,
      156.83939193719058,
      163.74155074212698,
      267.77352255903185,
      293.8815391540434,
      397.9134555311781,
      459.5323789291456,
      473.336677288115,
      569.9663454282004,
      583.7706250491924,
      619.7816885332018,
      626.2002727509848,
      639.5043977591581,
      714.9275810920541,
      729.7154903560316,
      743.5363366740661,
      757.8240689120161],
     'duration': [0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,
      0.001,


# -------- RANDOM CODE ---------

In [1]:
from nipype.utils.filemanip import loadpkl
res = loadpkl('/home/Public/trondheim/processing/crashdumps/crash-20220112-105134-scotti-flame1.b01-4d4679fc-4458-487d-8450-5b1fb1398176.pklz')
res

220113-11:12:31,251 nipype.utils WARNING:
	 A newer version (1.7.0) of nipy/nipype is available. You are using 1.6.0


{'node': feat_level3_msit.flame1.b01,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 516, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 635, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 741, in _run_command\n    result = self._interface.run(cwd=outdir)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 434, in run\n    runtime = self._run_interface(runtime)\n',
  '  File "/home/scotti/.c

In [24]:
res = loadpkl('/home/Public/trondheim/processing/crashdumps/crash-20220112-103613-scotti-flame1.b00-502f9333-1b41-4dbb-94a2-5d21ec861556.pklz')
res

{'node': feat_level3_msit.flame1.b00,
 'traceback': ['Traceback (most recent call last):\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/plugins/multiproc.py", line 67, in run_node\n    result["result"] = node.run(updatehash=updatehash)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 516, in run\n    result = self._run_interface(execute=True)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 635, in _run_interface\n    return self._run_command(execute)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/pipeline/engine/nodes.py", line 741, in _run_command\n    result = self._interface.run(cwd=outdir)\n',
  '  File "/home/scotti/.conda/envs/py38/lib/python3.8/site-packages/nipype/interfaces/base/core.py", line 434, in run\n    runtime = self._run_interface(runtime)\n',
  '  File "/home/scotti/.c